In [ ]:
!python -m pip install -U git+https://github.com/jerzydziewierz/mict.git#egg=mict

  Cloning https://github.com/jerzydziewierz/mict.git to /tmp/pip-install-e6j5x3d_/mict_2e6163b385c344a7b68c9d6ffaca1ace
  Running command git clone --filter=blob:none --quiet https://github.com/jerzydziewierz/mict.git /tmp/pip-install-e6j5x3d_/mict_2e6163b385c344a7b68c9d6ffaca1ace
  Resolved https://github.com/jerzydziewierz/mict.git to commit dd2c270975b8ada4d962738b69a391669f8bdcb0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for mict: filename=mict-0.0.41-py3-none-any.whl size=7997 sha256=9bc85b46e65be92f0448bcc060feed297a402bcc58e8009d856a0b7a19d7ca80
  Stored in directory: /tmp/pip-ephem-wheel-cache-i7jxj9ut/wheels/c5/b1/40/4c727cd22f94ce6f9be630eed4035b77f9b99823cbf453e2c3
Successfully built mict


In [ ]:
from mict import mict

In [ ]:
from functools import reduce
import numpy

# FAF ECO

this will simulate the economics of game: "Supreme commander Forged Alliance Forever"
with the express intent to figure out what is the fastest way to build a specific game-ender, e.g. mavor

the concept is as follows:
for each second:
* Building:
  * Sum the total build power, mass income, energy income
  * if build build is not finished, continue to build
  * if build is finished, select a new unit to build


Unit types. Data from the "FAF Launcher", UEF:
reference: https://faforever.github.io/spooky-db/#/


| Name | cost-buildpower | cost-mass | cost-energy | provides-buildpower | provides-mass | provides-energy |
| -- | -- | -- | -- | -- | -- | -- |
| COM | 0 | 0 | 0 | 10 | 1 | 20 |
| Factory T1 | ? | ? | ? | ? | ? | ? |
| Factory T2 | ? | ? | ? | ? | ? | ? |
| Factory T3 | ? | ? | ? | ? | ? | ? |
| Builder T1 | 260 | 52 | 260 | 5 | 0 | 0 |
| Builder T2 | 650 | 130 | 650 | 13 | 0 | 0 |
| Builder Sparky | 1000 | 225 | 1000 | 17.5 | 0 | 0 |
| Builder T3 | 1560 | 312 | 1560 | 32.5 | 0 | 0 |
| Build tower UEF | 1000 | 525 | 2625 | 20 | 0 | 0 |
| Mex T1 | 60 | 36 | 360 | 10 for upgrade only | 2 | -2 |
| Mex T2 | 900 | 900 | 5400 | 15 for upgrade only | 6 | 9 |
| Mex T3 | 2875 | 4600 | 31625 | 0 | 18 | -54 |
| Name | cost-buildpower | cost-mass | cost-energy | provides-buildpower | provides-mass | provides-energy |
| Powerplant T1 | 125 | 75 | 750 | 0 | 0 | 20 |
| Powerplant T2 | 2198 | 1200 | 12000 | 0 | 0 | 500 |
| Powerplant T3 | 6824 | 3240 | 57600 | 0 | 0 | 2500 |
| Name | cost-buildpower | cost-mass | cost-energy | provides-buildpower | provides-mass | provides-energy |
| MassGen T1 | 500 | 200 | 4000 | 0 | 1 | -100 |
| MassGen T2 | 5000 | 4000 | 120000 | 0 | 16 | -1500 |
| Name | cost-buildpower | cost-mass | cost-energy | provides-buildpower | provides-mass | provides-energy |
| Mavor | 300000 | 224775 | 5.994e6 | 0 | 0 | 0 |
| Novax | 44800 | 36000 | 512e3 | 0 | 0 | 0 |
The goal is to find the shortest path to building the Mavor.



In [ ]:
log = print

class Unit:
  def __init__(self):
    self.cost_buildpower = None
    self.cost_mass = None
    self.cost_energy = None
    self.provides_buildpower = 0
    self.provides_mass = 0
    self.provides_energy = 0
    self.mass_storage = 0
    self.energy_storage = 0

  @staticmethod
  def name():
    return "unit"

  def can_be_built(unit_list = []) -> bool:
    """
    Takes a list of current units available, and returns True if this can be built.
    """
    return False

  def on_build(unit_list = []) -> list:
    """
    when build is complete, takes a list of current units, and returns an updated list.
    For example, an unit can be created, converted, or consumed.
    """
    pass

  @classmethod
  def __str__(cls):
    return f'{cls.name()}'

  @classmethod
  def __repr__(cls):
    return f'{cls.name()}'

class Com(Unit):

  @staticmethod
  def name():
    return "com"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 0.0
    self.cost_mass = 0.0
    self.cost_energy = 0.0
    self.provides_buildpower = 10.0
    self.provides_mass = 1.0
    self.provides_energy = 10.0
    self.mass_storage = 650
    self.energy_storage = 3900

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    return False

  @staticmethod
  def on_build(unit_list = []) -> list:
    log('MexPlace cannot be built during the game')
    assert(False)

class Endgame(Unit):

  @staticmethod
  def name():
    return "endgame"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 1.0
    self.cost_mass = 0.0
    self.cost_energy = 0.0
    self.provides_buildpower = 0.0
    self.provides_mass = 0.0
    self.provides_energy = 00.0

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    exists = set(unit_list)
    to_build_require = {'mavor'}
    can_build = to_build_require.intersection(exists) == to_build_require
    return can_build

  @staticmethod
  def on_build(unit_list = []) -> list:
    unit_list.append(Endgame())
    log('game ended.')
    return unit_list

class MexPlace(Unit):

  @staticmethod
  def name():
    return "mexplace"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 0.0
    self.cost_mass = 0.0
    self.cost_energy = 0.0
    self.provides_buildpower = 0.0
    self.provides_mass = 0.0
    self.provides_energy = 00.0

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    exists = set(unit_list)
    to_build_require = {}
    can_build = to_build_require.intersection(exists) == to_build_require
    return can_build

  @staticmethod
  def on_build(unit_list = []) -> list:
    log('MexPlace cannot be built during the game')
    assert(False)


class Mavor(Unit):
  @staticmethod
  def name():
    return "mavor"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 300000.0
    self.cost_mass = 224775.0
    self.cost_energy = 5.994e6
    self.provides_buildpower = 0.0
    self.provides_mass = 0.0
    self.provides_energy = 00.0

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    exists = set(unit_list)
    to_build_require = {'com'}
    can_build = to_build_require.intersection(exists) == to_build_require
    return can_build

  @staticmethod
  def on_build(unit_list = []) -> list:
    unit_list.append(Mavor())
    return unit_list


class PowerplantT1(Unit):
  @staticmethod
  def name():
    return "powerplantT1"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 125.0
    self.cost_mass = 75.0
    self.cost_energy = 750
    self.provides_buildpower = 0.0
    self.provides_mass = 0.0
    self.provides_energy = 20.0

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    exists = set(unit_list)
    to_build_require = {'com'}
    can_build = to_build_require.intersection(exists) == to_build_require
    return can_build

  @staticmethod
  def on_build(unit_list = []) -> list:
    unit_list.append(PowerplantT1())
    return unit_list


class MexT1(Unit):
  @staticmethod
  def name():
    return "mexT1"

  def __init__(self):
    super().__init__()
    self.cost_buildpower = 60
    self.cost_mass = 36
    self.cost_energy = 360
    self.provides_buildpower = 0.0
    self.provides_mass = 2.0
    self.provides_energy = -2.0

  @staticmethod
  def can_be_built(unit_list = []) -> bool:
    exists_in_available = set(unit_list)
    to_build_require = {'com', 'mexplace'}
    # this verifies that all the items that are required, exist in the unit list
    can_build = to_build_require.intersection(exists_in_available) == to_build_require
    return can_build

  @staticmethod
  def on_build(unit_list = []) -> list:
    # remove one mexplace from the list
    is_first_mexplace = True
    filtered_list = [item for item in unit_list if not (item.name() == 'mexplace' and is_first_mexplace) or (is_first_mexplace := False)]
    filtered_list.append(MexT1())
    return filtered_list


In [ ]:
# note: we will deal with unit instances, not with types, to reduce amount of confusion
possible_unit_types = [Com(), Endgame(), MexPlace(), Mavor(), PowerplantT1(), MexT1()]

In [ ]:
possible_unit_types
for idx, unit in enumerate(possible_unit_types):
  print(idx, unit.name(), unit.provides_energy)

0 com 10.0
1 endgame 0.0
2 mexplace 0.0
3 mavor 0.0
4 powerplantT1 20.0
5 mexT1 -2.0


In [ ]:
def unit_name_list(unit_list: list = []) -> list:
  return [unit.name() for unit in unit_list]

def unit_type_from_name(unit_name = None):
  if unit_name is None:
    raise ValueError('cannot convert None to unit type')
  unl = unit_name_list(possible_unit_types)
  lookup_table = dict(list(zip(unl,possible_unit_types)))
  result = lookup_table[unit_name]
  return result

def test_unit_type_from_name():
  unit_type_from_name('com')

test_unit_type_from_name()

In [ ]:
def test_can_endgame_be_built():
  current_unit_list = list()
  current_unit_list.append(Com())

  for idx in range(8):
    current_unit_list.append(MexPlace())

  assert(Endgame.can_be_built(unit_name_list(current_unit_list))==False)
  current_unit_list.append(Mavor())
  assert(Endgame.can_be_built(unit_name_list(current_unit_list))==True)

test_can_endgame_be_built()

In [ ]:
def apply_next_buildorder(game_state, unit_to_build) -> mict:
  game_state.currently_building = unit_to_build
  game_state.currently_building_buildpower_progress = 0
  return game_state

In [ ]:
def select_next_buildorder(game_state: mict = mict(), local_state: mict=mict()):
  # this will need to be a bit smart
  # for now, I will use an algorithmic approach
  # if we can build endgame, do it
  # if the
  # if we can build a mex on a MexPlace, do it
  # then,
  # if there is less T1 powerplants than mexes of any kind, build T1 powerplant
  # then,
  # build mavor

  if Endgame.can_be_built(unit_name_list(game_state.current_unit_list)):
    return Endgame()

  if MexT1.can_be_built(unit_name_list(game_state.current_unit_list)):
    if local_state.energy_production_capability>5: # but do not build if there is not enough power.
      return MexT1()

  if local_state.energy_production_capability < 50:
    return PowerplantT1()
  else:
    return Mavor()
  # at this point, something must be selected.
  assert(False)

In [ ]:
def test_select_next_buildorder():
  assert("mavor" == str(select_next_buildorder(game_state=mict(current_unit_list=[]), local_state=mict(energy_production_capability=30000))))
  assert("powerplantT1" == str(select_next_buildorder(game_state=mict(current_unit_list=[]), local_state=mict(energy_production_capability=30))))
  assert(isinstance(select_next_buildorder(game_state=mict(current_unit_list=[Mavor()]), local_state=mict(energy_production_capability=30)), Endgame))
test_select_next_buildorder()

In [ ]:
def sum(x,y):
  return x+y

In [ ]:
def init_game_state():
  game_state = mict()
  game_state.current_unit_list = list()

  game_state.idx_gamecycle=0
  # add a com.
  game_state.current_unit_list.append(Com())
  # add 8 mex places - this simulates "dual gap" map.
  for idx in range(8):
    game_state.current_unit_list.append(MexPlace())

  # at first, nothing is building.
  game_state.currently_building = None
  game_state.currently_building_buildpower_progress = 0

  # init bank.

  game_state.energy_bank = 0
  game_state.mass_bank = 0
  game_state.energy_wasted = 0
  game_state.mass_wasted = 0

  # the game starts with "full bank". Although, I observe that in the real game, the starting energy is 4000, not just Com's 3600.

  # game initializes with full bank:
  game_state.mass_bank = Com().mass_storage
  game_state.energy_bank = Com().energy_storage
  return game_state

log_cycle = False

In [ ]:
def game_turn(game_state: mict = None) -> mict:

  return game_state

game_state = init_game_state()

In [ ]:
for idx_minicycle in range(10000):
  idx_gamecycle +=1
  # add energy and mass to the bank, up to the capacity
  mass_production_capability = reduce(sum, [unit.provides_mass for unit in game_state.current_unit_list])
  energy_production_capability = reduce(sum, [unit.provides_energy for unit in game_state.current_unit_list])
  build_power_capability = reduce(sum, [unit.provides_buildpower for unit in game_state.current_unit_list])

  mass_storage_capacity = reduce(sum, [unit.mass_storage for unit in game_state.current_unit_list])
  energy_storage_capacity = reduce(sum, [unit.energy_storage for unit in game_state.current_unit_list])


  game_state.mass_bank = game_state.mass_bank + mass_production_capability
  game_state.mass_wasted = game_state.mass_wasted + numpy.maximum(0,game_state.mass_bank-mass_storage_capacity);
  game_state.mass_bank = numpy.minimum(mass_storage_capacity,game_state.mass_bank)

  game_state.energy_bank = game_state.energy_bank + energy_production_capability
  game_state.energy_wasted = game_state.energy_wasted + numpy.maximum(0.0,game_state.energy_bank-energy_storage_capacity)
  game_state.energy_bank = numpy.minimum(energy_storage_capacity,game_state.energy_bank)

  local_state = mict.from_locals()

  # if not building, continue:
  if game_state.currently_building is None:
    next_unit = select_next_buildorder(game_state, local_state)
    game_state = apply_next_buildorder(game_state, next_unit)
    log_cycle = True

  # approach:
  # given current build power, energy_bank, mass_bank
  # how much build power can be spent?
  # note that this does not come from the game's source code, but rather, is my recreation according to the observations.
  # it appears to function correctly.
  mass_per_bp = game_state.currently_building.cost_mass / game_state.currently_building.cost_buildpower
  energy_per_bp  = game_state.currently_building.cost_energy / game_state.currently_building.cost_buildpower

  mass_spend_proposal = build_power_capability * mass_per_bp
  energy_spend_proposal = build_power_capability * energy_per_bp

  bank_limited_mass_spend = numpy.minimum(mass_spend_proposal,game_state.mass_bank)
  bank_limited_energy_spend = numpy.minimum(energy_spend_proposal,game_state.energy_bank)

  bp_mass_spend_limited = bank_limited_mass_spend / mass_per_bp
  bp_energy_spend_limited = bank_limited_energy_spend / energy_per_bp

  actual_bp_usable = numpy.minimum(bp_mass_spend_limited,bp_energy_spend_limited)

  actual_mass_used = actual_bp_usable * mass_per_bp
  actual_energy_used = actual_bp_usable * energy_per_bp

  # apply
  game_state.currently_building_buildpower_progress+=actual_bp_usable
  game_state.mass_bank -= actual_mass_used
  game_state.energy_bank -= actual_energy_used

  # if build finished:
  if game_state.currently_building_buildpower_progress > game_state.currently_building.cost_buildpower:
    game_state.current_unit_list = game_state.currently_building.on_build(game_state.current_unit_list)
    game_state.currently_building = None
    build_cycles_remaining_estimate = numpy.nan
  else:
    build_cycles_remaining_estimate = (game_state.currently_building.cost_buildpower-game_state.currently_building_buildpower_progress)/actual_bp_usable


  if (idx_gamecycle % 1000 == 0) and game_state.currently_building is not None:
    log_cycle = True

  if log_cycle:
    log(f'cycle {idx_gamecycle} : building {game_state.currently_building} ({game_state.currently_building_buildpower_progress:0.0f} @ {actual_bp_usable:0.0f} : {build_cycles_remaining_estimate:0.0f}),  PC={energy_production_capability:+0.0f}/{mass_production_capability:+0.0f}/{build_power_capability:+0.0f}, bank = {game_state.energy_bank:0.0f}/{game_state.mass_bank:0.0f}')
    log_cycle = False

  # estimate stats





cycle 20001 : building mexT1 (10 @ 10 : 5),  PC=+10/+1/+10, bank = 3840/644
cycle 20008 : building mexT1 (10 @ 10 : 5),  PC=+8/+3/+10, bank = 3488/611
cycle 20015 : building mexT1 (10 @ 10 : 5),  PC=+6/+5/+10, bank = 3122/592
cycle 20022 : building powerplantT1 (10 @ 10 : 12),  PC=+4/+7/+10, bank = 2742/587
cycle 20035 : building mexT1 (10 @ 10 : 5),  PC=+24/+7/+10, bank = 2034/600
cycle 20042 : building mexT1 (10 @ 10 : 5),  PC=+22/+9/+10, bank = 1780/609
cycle 20049 : building mexT1 (10 @ 10 : 5),  PC=+20/+11/+10, bank = 1512/632
cycle 20056 : building mexT1 (10 @ 10 : 5),  PC=+18/+13/+10, bank = 1230/644
cycle 20063 : building mexT1 (10 @ 10 : 5),  PC=+16/+15/+10, bank = 934/644
cycle 20070 : building powerplantT1 (10 @ 10 : 12),  PC=+14/+17/+10, bank = 624/644
cycle 20083 : building powerplantT1 (10 @ 10 : 12),  PC=+34/+17/+10, bank = 46/644
cycle 20103 : building mavor (3 @ 3 : 110999),  PC=+54/+17/+10, bank = 0/648
cycle 21000 : building mavor (2427 @ 3 : 110102),  PC=+54/+17/+10

In [ ]:
current_unit_list_names